In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools

## Read Data

In [4]:
# true: local cv, false: test
valid_flag = True
#valid_flag = False

base_path = '/content/drive/MyDrive/input/otto'
output_path = '/content/drive/MyDrive/output/otto'
if valid_flag:
  input_path = base_path + '/otto-validation'
else:
  input_path = base_path + '/otto-origin'

In [5]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_data():    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(f'{input_path}/*_parquet/*')):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

all_df = load_data()
# reduce memory
all_df['session'] = all_df['session'].astype('int32')
all_df['aid'] = all_df['aid'].astype('int32')

print('All data has shape',all_df.shape)
all_df.head()
print('All:', (all_df['ts'].max() - all_df['ts'].min()) / 60 / 60 / 24, 'days')

All data has shape (171638757, 4)
All: 27.999953703703707 days


In [6]:
all_period_ts_max = all_df['ts'].max()
ts_1day = 60 * 60 * 24
ts_1week = ts_1day * 7
ts_2weeks = ts_1day * 7 * 2
#ts_3weeks = ts_1day * 7 * 3
ts_4weeks = ts_1day * 7 * 4
#week_list = ['4weeks', '3weeks', '2weeks', '1week']
#ts_list = [ts_4weeks, ts_3weeks, ts_2weeks, ts_1week]
week_list = ['4weeks', '2weeks', '1week']
ts_list = [ts_4weeks, ts_2weeks, ts_1week]

In [7]:
all_df['ts'].max()

1661723996

In [8]:
aid_df = pd.DataFrame(all_df['aid'].unique(), columns=["aid"])
#session_df = pd.DataFrame(all_df['session'].unique(), columns=["session"])
#print('aid shape:', aid_df.shape, 'session shape:',session_df.shape)
print('aid shape:', aid_df.shape)

aid shape: (1844284, 1)


In [9]:
def type_change(df, column_name, num, change_type):
    df[column_name] = df[column_name].fillna(num).astype(change_type)
    return df

In [10]:
all_df.head()

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0


## Add aid features

In [11]:
Ntop_all = 1000000

df = all_df.copy()

for i, t in zip(week_list, ts_list):
    print('i=',i,'*******')
    # start from longer period
    df = df[all_period_ts_max - df['ts'] < t]
    print(df.shape)
    print('get aid feature by count....')
    # click, cart, order count for each aid
    top_clicks_num_df = df.loc[df['type']== 0,'aid'].value_counts()
    top_carts_num_df = df.loc[df['type']== 1,'aid'].value_counts()
    top_orders_num_df = df.loc[df['type']== 2,'aid'].value_counts()

    top_clicks_num_df = pd.DataFrame({'aid': top_clicks_num_df.index, f'aid_clicks_count_{i}': top_clicks_num_df.values})
    top_carts_num_df = pd.DataFrame({'aid': top_carts_num_df.index, f'aid_carts_count_{i}': top_carts_num_df.values})
    top_orders_num_df = pd.DataFrame({'aid': top_orders_num_df.index, f'aid_orders_count_{i}': top_orders_num_df.values})
    
    top_counts_df = top_clicks_num_df.merge(top_carts_num_df, how = 'outer', on = 'aid').merge(top_orders_num_df, how = 'outer', on = 'aid').fillna(0)
    del top_clicks_num_df, top_carts_num_df, top_orders_num_df
    gc.collect()

    top_counts_df['aid'] = top_counts_df['aid'].astype('int32')
    aid_df = aid_df.merge(top_counts_df, how = 'left', on = ['aid'])
    del top_counts_df
    gc.collect()

    aid_df[f'aid_clicks_count_{i}'] = aid_df[f'aid_clicks_count_{i}'].fillna(0).astype('int32')
    aid_df[f'aid_carts_count_{i}'] = aid_df[f'aid_carts_count_{i}'].fillna(0).astype('int16')
    aid_df[f'aid_orders_count_{i}'] = aid_df[f'aid_orders_count_{i}'].fillna(0).astype('int16')
    aid_df[f'aid_total_count_{i}'] = (aid_df[f'aid_clicks_count_{i}'] + aid_df[f'aid_carts_count_{i}'] + aid_df[f'aid_orders_count_{i}']).astype('int32')
    
    print('get aid feature by uu....')
    # unique user (uu) count for each aid
    total_uu = df.groupby('aid')['session'].nunique()
    clicks_uu = df[df['type']== 0].groupby('aid')['session'].nunique()
    carts_uu = df[df['type']== 1].groupby('aid')['session'].nunique()
    orders_uu =df[df['type']== 2].groupby('aid')['session'].nunique()

    total_uu = pd.DataFrame({'aid': total_uu.index, f'aid_total_uu_{i}': total_uu.values})
    clicks_uu = pd.DataFrame({'aid': clicks_uu.index, f'aid_clicks_uu_{i}': clicks_uu.values})
    carts_uu = pd.DataFrame({'aid': carts_uu.index, f'aid_carts_uu_{i}': carts_uu.values})
    orders_uu = pd.DataFrame({'aid': orders_uu.index, f'aid_orders_uu_{i}': orders_uu.values})

    aid_uu_df = total_uu.merge(clicks_uu, how = 'outer', on = 'aid').merge(carts_uu, how = 'outer', on = 'aid').merge(orders_uu, how = 'outer', on = 'aid').fillna(0)
    aid_uu_df['aid'] = aid_uu_df['aid'].astype('int32')
    aid_df = aid_df.merge(aid_uu_df, how = 'left', on = ['aid'])
    del total_uu, clicks_uu, carts_uu, orders_uu
    gc.collect()

    aid_df[f'aid_total_uu_{i}'] = aid_df[f'aid_total_uu_{i}'].fillna(0).astype('int32')
    aid_df[f'aid_clicks_uu_{i}'] = aid_df[f'aid_clicks_uu_{i}'].fillna(0).astype('int32')
    aid_df[f'aid_carts_uu_{i}'] = aid_df[f'aid_carts_uu_{i}'].fillna(0).astype('int32')
    aid_df[f'aid_orders_uu_{i}'] = aid_df[f'aid_orders_uu_{i}'].fillna(0).astype('int32')
    
    # uu/(action count) ratio. if this value is mall, small user have a lot of actions
    aid_df[f'aid_total_uu_action_ratio_{i}'] = aid_df[f'aid_total_uu_{i}'] / (aid_df[f'aid_total_count_{i}'] + 0.000001)
    aid_df[f'aid_clicks_uu_action_ratio_{i}'] = aid_df[f'aid_clicks_uu_{i}'] / (aid_df[f'aid_clicks_count_{i}'] + 0.000001)
    aid_df[f'aid_carts_uu_action_ratio_{i}'] = aid_df[f'aid_carts_uu_{i}'] / (aid_df[f'aid_carts_count_{i}'] + 0.000001)
    aid_df[f'aid_orders_uu_action_ratio_{i}'] = aid_df[f'aid_orders_uu_{i}'] / (aid_df[f'aid_orders_count_{i}'] + 0.000001)

    aid_df[f'aid_total_uu_action_ratio_{i}']  = aid_df[f'aid_total_uu_action_ratio_{i}'].astype('float32')  
    aid_df[f'aid_clicks_uu_action_ratio_{i}'] = aid_df[f'aid_clicks_uu_action_ratio_{i}'].astype('float32') 
    aid_df[f'aid_carts_uu_action_ratio_{i}']  = aid_df[f'aid_carts_uu_action_ratio_{i}'].astype('float32')
    aid_df[f'aid_orders_uu_action_ratio_{i}'] = aid_df[f'aid_orders_uu_action_ratio_{i}'].astype('float32')
    
    print('get aid feature by ranking....')
    # rank calculation
    top_clicks = df.loc[df['type']== 0,'aid'].value_counts().index.values[:Ntop_all] 
    top_carts = df.loc[df['type']== 1,'aid'].value_counts().index.values[:Ntop_all]
    top_orders = df.loc[df['type']== 2,'aid'].value_counts().index.values[:Ntop_all]

    dic_clicks = dict()
    dic_carts = dict()
    dic_orders = dict()

    for j, v in enumerate(top_clicks):
        dic_clicks[v] = j
    for j, v in enumerate(top_carts):
        dic_carts[v] = j
    for j, v in enumerate(top_orders):
        dic_orders[v] = j
    del top_clicks, top_carts, top_orders
    gc.collect()

    # click, carts, order ranking
    aid_df[f'aid_clicks_rank_{i}'] = aid_df['aid'].apply(lambda x: dic_clicks[x] if x in dic_clicks else -1).astype('int32')
    aid_df[f'aid_carts_rank_{i}'] = aid_df['aid'].apply(lambda x: dic_carts[x] if x in dic_carts else -1).astype('int32')
    aid_df[f'aid_orders_rank_{i}'] = aid_df['aid'].apply(lambda x: dic_orders[x] if x in dic_orders else -1).astype('int32')
    del dic_clicks, dic_carts, dic_orders
    gc.collect()

    # session mean action/click/cart/order used for aid feature
    print('get aid feature by session info....')

    # total action
    tmp_session_df = df.groupby('session').size().reset_index(name=f'tmp_session_action_count_{i}')
    tmp_session_df = type_change(tmp_session_df, f'tmp_session_action_count_{i}', 0, 'int16')
    # click
    tmp = df.query('type==0').groupby('session').size().reset_index(name=f'tmp_session_click_count_{i}')
    tmp_session_df = tmp_session_df.merge(tmp, how = 'left', on = 'session')
    tmp_session_df = type_change(tmp_session_df, f'tmp_session_click_count_{i}', 0, 'int16')
    # cart
    tmp = df.query('type==1').groupby('session').size().reset_index(name=f'tmp_session_cart_count_{i}')
    tmp_session_df = tmp_session_df.merge(tmp, how = 'left', on = 'session')
    tmp_session_df = type_change(tmp_session_df, f'tmp_session_cart_count_{i}', 0, 'int16')
    # order
    tmp = df.query('type==2').groupby('session').size().reset_index(name=f'tmp_session_order_count_{i}')
    tmp_session_df = tmp_session_df.merge(tmp, how = 'left', on = 'session')
    tmp_session_df = type_change(tmp_session_df, f'tmp_session_order_count_{i}', 0, 'int16')
    # type_mean
    tmp = df.groupby('session').mean()['type'].reset_index(name=f'tmp_session_type_mean_{i}')
    tmp_session_df = tmp_session_df.merge(tmp, how = 'left', on = 'session')
    tmp_session_df = type_change(tmp_session_df, f'tmp_session_type_mean_{i}', -1, 'float32')

    df = df.merge(tmp_session_df, how = 'left', on = 'session')
    del tmp, tmp_session_df
    gc.collect()

    aid_df[f'aid_mean_session_action_count_{i}'] = df.groupby('aid')[f'tmp_session_action_count_{i}'].mean()
    aid_df[f'aid_mean_session_click_count_{i}'] = df.groupby('aid')[f'tmp_session_click_count_{i}'].mean()
    aid_df[f'aid_mean_session_cart_count_{i}'] = df.groupby('aid')[f'tmp_session_cart_count_{i}'].mean()
    aid_df[f'aid_mean_session_order_count_{i}'] = df.groupby('aid')[f'tmp_session_order_count_{i}'].mean()
    aid_df[f'aid_mean_session_type_mean_{i}'] = df.groupby('aid')[f'tmp_session_type_mean_{i}'].mean()

    aid_df = type_change(aid_df, f'aid_mean_session_action_count_{i}', 0, 'float32')
    aid_df = type_change(aid_df, f'aid_mean_session_click_count_{i}', 0, 'float32')
    aid_df = type_change(aid_df, f'aid_mean_session_cart_count_{i}', 0, 'float32')
    aid_df = type_change(aid_df, f'aid_mean_session_order_count_{i}', 0, 'float32')
    aid_df = type_change(aid_df, f'aid_mean_session_type_mean_{i}', 0, 'float32')


i= 4weeks *******
(171638757, 4)
get aid feature by count....
get aid feature by uu....
get aid feature by ranking....
get aid feature by session info....
i= 2weeks *******
(63090916, 9)
get aid feature by count....
get aid feature by uu....
get aid feature by ranking....
get aid feature by session info....
i= 1week *******
(7683780, 14)
get aid feature by count....
get aid feature by uu....
get aid feature by ranking....
get aid feature by session info....


In [12]:
# ratio feature based on week
for i in ['clicks', 'carts', 'orders']:
    for j in [2,4]:
        aid_df[f'aid_{i}_count_rate_1_{j}'] = aid_df[f'aid_{i}_count_1week'] / (aid_df[f'aid_{i}_count_{j}weeks'] + 0.000001)
        aid_df[f'aid_{i}_count_rate_1_{j}'] = aid_df[f'aid_{i}_count_rate_1_{j}'].astype('float32')

        aid_df[f'aid_{i}_uu_rate_1_{j}'] = aid_df[f'aid_{i}_uu_1week'] / (aid_df[f'aid_{i}_uu_{j}weeks'] + 0.000001)
        aid_df[f'aid_{i}_uu_rate_1_{j}'] = aid_df[f'aid_{i}_uu_rate_1_{j}'].astype('float32')
        # only first i loop
        if i == 'clicks':
            aid_df[f'aid_total_uu_rate_1_{j}'] = aid_df[f'aid_total_uu_1week'] / (aid_df[f'aid_total_uu_{j}weeks'] + 0.000001)
            aid_df[f'aid_total_uu_rate_1_{j}'] = aid_df[f'aid_total_uu_rate_1_{j}'].astype('float32')

In [13]:
pd.set_option('display.max_columns', 100)
aid_df

,aid,aid_clicks_count_4weeks,aid_carts_count_4weeks,aid_orders_count_4weeks,aid_total_count_4weeks,aid_total_uu_4weeks,aid_clicks_uu_4weeks,aid_carts_uu_4weeks,aid_orders_uu_4weeks,aid_total_uu_action_ratio_4weeks,aid_clicks_uu_action_ratio_4weeks,aid_carts_uu_action_ratio_4weeks,aid_orders_uu_action_ratio_4weeks,aid_clicks_rank_4weeks,aid_carts_rank_4weeks,aid_orders_rank_4weeks,aid_mean_session_action_count_4weeks,aid_mean_session_click_count_4weeks,aid_mean_session_cart_count_4weeks,aid_mean_session_order_count_4weeks,aid_mean_session_type_mean_4weeks,aid_clicks_count_2weeks,aid_carts_count_2weeks,aid_orders_count_2weeks,aid_total_count_2weeks,aid_total_uu_2weeks,aid_clicks_uu_2weeks,aid_carts_uu_2weeks,aid_orders_uu_2weeks,aid_total_uu_action_ratio_2weeks,aid_clicks_uu_action_ratio_2weeks,aid_carts_uu_action_ratio_2weeks,aid_orders_uu_action_ratio_2weeks,aid_clicks_rank_2weeks,aid_carts_rank_2weeks,aid_orders_rank_2weeks,aid_mean_session_action_count_2weeks,aid_mean_session_click_count_2weeks,aid_mean_session_cart_count_2weeks,aid_mean_session_order_count_2weeks,aid_mean_session_type_mean_2weeks,aid_clicks_count_1week,aid_carts_count_1week,aid_orders_count_1week,aid_total_count_1week,aid_total_uu_1week,aid_clicks_uu_1week,aid_carts_uu_1week,aid_orders_uu_1week,aid_total_uu_action_ratio_1week,aid_clicks_uu_action_ratio_1week,aid_carts_uu_action_ratio_1week,aid_orders_uu_action_ratio_1week,aid_clicks_rank_1week,aid_carts_rank_1week,aid_orders_rank_1week,aid_mean_session_action_count_1week,aid_mean_session_click_count_1week,aid_mean_session_cart_count_1week,aid_mean_session_order_count_1week,aid_mean_session_type_mean_1week,aid_clicks_count_rate_1_2,aid_clicks_uu_rate_1_2,aid_total_uu_rate_1_2,aid_clicks_count_rate_1_4,aid_clicks_uu_rate_1_4,aid_total_uu_rate_1_4,aid_carts_count_rate_1_2,aid_carts_uu_rate_1_2,aid_carts_count_rate_1_4,aid_carts_uu_rate_1_4,aid_orders_count_rate_1_2,aid_orders_uu_rate_1_2,aid_orders_count_rate_1_4,aid_orders_uu_rate_1_4
0,1517085,85,13,3,101,58,58,11,2,0.574257,0.682353,0.846154,0.666666,257959,173381,194587,72.236839,63.921051,5.447369,2.868421,0.173853,20,2,0,22,13,13,2,0,0.590909,0.650000,1.000000,0.000000,367426,325485,-1,40.933334,38.266666,1.800000,0.866667,0.099908,6,1,0,7,4,4,1,0,0.571428,0.666667,0.999999,0.000000,200249,223414,-1,65.000000,61.400002,2.800000,0.80000,0.127259,0.300000,0.307692,0.307692,0.070588,0.068966,0.068966,0.500000,0.500000,0.076923,0.090909,0.000000,0.000000,0.000000,0.000000
1,1563459,74,0,0,74,71,71,0,0,0.959459,0.959459,0.000000,0.000000,286867,-1,-1,70.312500,65.812500,3.937500,0.562500,0.059889,26,0,0,26,25,25,0,0,0.961538,0.961538,0.000000,0.000000,305151,-1,-1,90.500000,84.250000,5.000000,1.250000,0.066792,10,0,0,10,9,9,0,0,0.900000,0.900000,0.000000,0.000000,113772,-1,-1,0.000000,0.000000,0.000000,0.00000,0.000000,0.384615,0.360000,0.360000,0.135135,0.126761,0.126761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1309446,4263,505,89,4857,2382,2374,356,77,0.490426,0.556885,0.704951,0.865169,3123,1943,5458,32.687500,30.625000,2.000000,0.062500,0.034980,1572,166,37,1775,930,928,135,32,0.523944,0.590331,0.813253,0.864865,3144,2325,4130,15.000000,15.000000,0.000000,0.000000,0.000000,148,12,2,162,106,106,12,2,0.654321,0.716216,1.000000,1.000000,4984,6948,9149,22.500000,22.500000,0.000000,0.00000,0.000000,0.094148,0.114224,0.113978,0.034717,0.044650,0.044500,0.072289,0.088889,0.023762,0.033708,0.054054,0.062500,0.022472,0.025974
3,16246,1095,120,39,1254,662,661,93,39,0.527911,0.603653,0.775000,1.000000,21036,15821,15985,33.049179,30.264917,2.161311,0.622951,0.089579,455,55,21,531,293,291,46,21,0.551789,0.639560,0.836364,1.000000,18074,11878,9263,22.775333,20.952667,1.453333,0.369333,0.087721,62,3,1,66,47,46,3,1,0.712121,0.741935,1.000000,0.999999,15703,54304,25534,10.700617,9.441358,1.138889,0.12037,0.093608,0.136264,0.158076,0.160410,0.056621,0.069592,0.070997,0.054545,0.065217,0.025000,0.032258,0.047619,0.047619,0.025641,0.025641
4,

## Save aid features

In [ ]:
if valid_flag:
    aid_df.to_parquet(f'{output_path}/valid_aid_features.parquet')
else:
    aid_df.to_parquet(f'{output_path}/test_aid_features.parquet')

## Add session features

In [14]:
df = all_df.copy()
#del all_df
#gc.collect()

# only last 1 week for session feature
i = week_list[-1]
t = ts_list[-1]

df = df[all_period_ts_max - df['ts'] < t]
df['ts_diff'] = all_period_ts_max - df['ts'] 
print(df.shape)

# sessionごとにtype別、全actions数を求める
# total action
session_df = df.groupby('session').size().reset_index(name='session_action_count')
session_df['session'] = session_df['session'].astype('int32')
session_df = type_change(session_df, 'session_action_count', 0, 'int16')

# click
tmp = df.query('type==0').groupby('session').size().reset_index(name='session_click_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_click_count', 0, 'int16')
# cart
tmp = df.query('type==1').groupby('session').size().reset_index(name='session_cart_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_cart_count', 0, 'int16')
# order
tmp = df.query('type==2').groupby('session').size().reset_index(name='session_order_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_order_count', 0, 'int16')
# type_mean
tmp = df.groupby('session').mean()['type'].reset_index(name='session_type_mean')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_type_mean', -1, 'float32')
# last action type
tmp = df.groupby('session').last()['type'].reset_index(name='session_last_type')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_type_mean', -1, 'int8') # actually no null value

# time diff, calculated from the latest ts
tmp = df.groupby('session').max()['ts_diff'].reset_index(name='session_first_action_ts_diff')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_first_action_ts_diff', -1, 'int32') # actually no null value

tmp = df.groupby('session').min()['ts_diff'].reset_index(name='session_last_action_ts_diff')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_last_action_ts_diff', -1, 'int32') # actually no null value

session_df['session_ts_period'] = session_df['session_first_action_ts_diff'] - session_df['session_last_action_ts_diff']

tmp = df.groupby('session').mean()['ts_diff'].reset_index(name='session_mean_action_ts_diff')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_mean_action_ts_diff', -1, 'float32') # actually no null value


# unique aid total/click/cart/order number of actions group by session
tmp = df.groupby('session')['aid'].nunique().reset_index(name='session_unique_aid_action_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_unique_aid_action_count', 0, 'int16')
# unique aid click
tmp = df[df['type']== 0].groupby('session')['aid'].nunique().reset_index(name='session_unique_aid_click_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_unique_aid_click_count', 0, 'int16')
# unique aid cart
tmp = df[df['type']== 1].groupby('session')['aid'].nunique().reset_index(name='session_unique_aid_cart_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_unique_aid_cart_count', 0, 'int16')
# unique aid order
tmp = df[df['type']== 2].groupby('session')['aid'].nunique().reset_index(name='session_unique_aid_order_count')
session_df = session_df.merge(tmp, how = 'left', on = 'session')
session_df = type_change(session_df, 'session_unique_aid_order_count', 0, 'int16')

# count ratio, (click/cart/order) / total action count
session_df['session_click_rate'] = session_df['session_click_count'] / (session_df['session_action_count'] + 0.000001)
session_df['session_cart_rate'] = session_df['session_cart_count'] / (session_df['session_action_count'] + 0.000001)
session_df['session_order_rate'] = session_df['session_order_count'] / (session_df['session_action_count'] + 0.000001)
    
session_df['session_click_rate'] = session_df['session_click_rate'].astype('float32')
session_df['session_cart_rate'] = session_df['session_cart_rate'].astype('float32')
session_df['session_order_rate'] = session_df['session_order_rate'].astype('float32')

# unique count ratio, (click/cart/order) / total action count
session_df['session_unique_aid_click_rate'] = session_df['session_unique_aid_click_count'] / (session_df['session_unique_aid_action_count'] + 0.000001)
session_df['session_unique_aid_cart_rate'] = session_df['session_unique_aid_cart_count'] / (session_df['session_unique_aid_action_count'] + 0.000001)
session_df['session_unique_aid_order_rate'] = session_df['session_unique_aid_order_count'] / (session_df['session_unique_aid_action_count'] + 0.000001)
    
session_df['session_unique_aid_click_rate'] = session_df['session_unique_aid_click_rate'].astype('float32')
session_df['session_unique_aid_cart_rate'] = session_df['session_unique_aid_cart_rate'].astype('float32')
session_df['session_unique_aid_order_rate'] = session_df['session_unique_aid_order_rate'].astype('float32')

# uu/(action count) ratio. if this value is mall, small user have a lot of actions
session_df['session_total_uu_action_ratio'] = session_df['session_unique_aid_action_count'] / (session_df['session_action_count'] + 0.000001)
session_df['session_clicks_uu_action_ratio'] = session_df['session_unique_aid_click_count'] / (session_df['session_click_count'] + 0.000001)
session_df['session_carts_uu_action_ratio'] = session_df['session_unique_aid_cart_count'] / (session_df['session_cart_count'] + 0.000001)
session_df['session_orders_uu_action_ratio'] = session_df['session_unique_aid_order_count'] / (session_df['session_order_count'] + 0.000001)

session_df['session_total_uu_action_ratio'] = session_df['session_total_uu_action_ratio'].astype('float32')
session_df['session_clicks_uu_action_ratio'] = session_df['session_clicks_uu_action_ratio'].astype('float32')
session_df['session_carts_uu_action_ratio'] = session_df['session_carts_uu_action_ratio'].astype('float32')
session_df['session_orders_uu_action_ratio'] = session_df['session_orders_uu_action_ratio'].astype('float32')

del tmp
gc.collect()


<ipython-input-14-d404325143f6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ts_diff'] = all_period_ts_max - df['ts']


(7683780, 5)


6

In [15]:
session_df.shape

(1801451, 25)

In [ ]:
# session based aid feature
session_aid = df.merge(aid_df, 'left', 'aid')
for i in week_list:
    print(i, "*******")
    lis = [f'aid_clicks_count_{i}', 
           f'aid_carts_count_{i}', 
           f'aid_orders_count_{i}', 
           f'aid_total_count_{i}', 
           f'aid_total_uu_{i}', 
           f'aid_clicks_uu_{i}', 
           f'aid_carts_uu_{i}', 
           f'aid_orders_uu_{i}',
           f'aid_total_uu_action_ratio_{i}',
           f'aid_clicks_uu_action_ratio_{i}',
           f'aid_carts_uu_action_ratio_{i}',
           f'aid_orders_uu_action_ratio_{i}']
    for l in lis:
        print(l)
        tmp = session_aid.groupby('session').mean()[l].reset_index(name=f'session_mean_{l}')
        session_df = session_df.merge(tmp, how = 'left', on = 'session')
        session_df[f'session_mean_{l}'] = session_df[f'session_mean_{l}'].astype('float32')

4weeks *******
aid_clicks_count_4weeks


## Save session features

In [ ]:
if valid_flag:
    session_df.to_parquet(f'{output_path}/valid_session_features.parquet')
else:
    session_df.to_parquet(f'{output_path}/test_session_features.parquet')